This notebook will show how to calibrate a Normal Hydrogen reference Electrode (NHE) in alkaline environment (pH larger than 8).

In [ ]:
import pandas as pd
import plotly.express as px
from scipy.signal import savgol_filter

Now it is time to load the data. This repository contains a .csv with data from a Cyclic Voltammetry scan of platinum wire vs. platinum wire with a NHE as reference electrode, performed in 1M KOH at room temperature. Ph = 14.

The parameters of the recorded CV scan was:
    init_voltage=0.45, # V
    final_voltage=0.45, # V
    apex1=1.4, # V
    apex2=0, # V
    stepsize=0.001, # Resolution in Volts
    scanrate=1, # V/s
    cycles=20, # We will use scan cycle 19

In [ ]:
# Load the potentiostat data from .csv file with headers
csv_file_path = 'data.csv'

# Define the name of the columns containing the current (I) and potential (V) data
current_column = 'Current (A)'
potential_column = 'Potential (V)'

# Ohmic resistance and ohmic correction
I_WANT_OHMIC_CORRECTION = True
OHMIC_CORRECTION_FACTOR = 0.95 # %
OHMIC_RESISTANCE = 0.5 # Ohm

In [ ]:
df_pt = pd.read_csv(csv_file_path, header=0)

In [ ]:
def set_column_headers_cv(df: pd.DataFrame) -> pd.DataFrame:
    """Set column headers for CV data

    Args:
        data (pd.DataFrame): Dataframe containing CV data in the order
        [Index, Time (s), Potential (WE vs. RHE) [V], Vu (V), Current [A],
        Vsig, Ach (V), IERange, Overbit1, Stop Test, Scan cycle, Temperature (C)]

    Returns:
        pd.DataFrame: Dataframe with new column headers
    """

    # Define the desired column names
    column_names = [
        "Time (s)",
        "Potential (WE vs. RHE) [V]",
        "Vu (V)",
        "Current [A]",
        "Vsig",
        "Ach (V)",
        "IERange",
        "Overbit1",
        "Stop Test",
        "Scan cycle",
        "Temperature (C)",
    ]

    # Check if the number of columns is 12 and insert "Index" at the beginning of the column names
    if len(df.columns) == 12:
        column_names.insert(0, "Index")

    # Assign the new column names to the dataframe
    df.columns = column_names

    # Specify the columns to drop from the dataframe
    columns_to_drop = [
        "Time (s)",
        "Vu (V)",
        "Vsig",
        "Ach (V)",
        "IERange",
        "Overbit1",
        "Stop Test",
        "Temperature (C)",
    ]

    # Drop the specified columns from the dataframe, ignoring any errors if columns are not present
    df = df.drop(columns=columns_to_drop, errors="ignore")

    # If "Index" column exists in the dataframe, drop it as well
    if "Index" in df.columns:
        df = df.drop(columns=["Index"])

    return df

In [ ]:
def filter_data(
    df: pd.DataFrame,
    lower_potential: float = 0.65,
    upper_potential: float = 0.85,
    first_cycle: int = 10,
    last_cycle: int = 19,
    apply_row_removal_for_prettyness = True,
) -> pd.DataFrame:
    """Filters a dataframe containing a cyclic voltammetry scan of a Pt catalyst with many cycles.


    Args:
        data (pd.DataFrame): Dataframe containing the data to filter. Must contain a
        the columns "Scan cycle" and "Corrected potential (WE vs. RHE) [V]".
        lower_potential (float, optional): Lower potential to select data from. Defaults to 0.7.
        upper_potential (float, optional): Upper potential to select data to. Defaults to 0.93.
        first_cycle (int, optional): First scan cycle to select data from. Defaults to 10.
        last_cycle (int, optional): Last scan cycle to select data to. Defaults to 19.

    Returns:
        pd.DataFrame: Dataframe with filtered data
    """
    print("Selecting part of the platinum CV scan to find peak")
    # Select data that for "Scan cycle" column in range (first_cycle, last_cycle)
    filtered_data = df[(df["Scan cycle"] >= first_cycle) & (df["Scan cycle"] <= last_cycle)]

    # Select data only within specified "Corrected potential (WE vs. RHE) [V]"
    filtered_data = filtered_data[
        (filtered_data["Corrected potential (WE vs. RHE) [V]"] >= lower_potential)
        & (filtered_data["Corrected potential (WE vs. RHE) [V]"] <= upper_potential)
    ]

    # Create a mask to check if the "Corrected potential" values are increasing
    mask = filtered_data["Corrected potential (WE vs. RHE) [V]"] > filtered_data[
        "Corrected potential (WE vs. RHE) [V]"
    ].shift(1)

    # Apply the mask to select the rows where the "Corrected potential" increases
    filtered_data = filtered_data[mask]

    if apply_row_removal_for_prettyness is True:
        # For each "Scan cycle", remove the last row of filtered_data to
        # avoid negative "Current" values
        filtered_data = filtered_data.groupby("Scan cycle").apply(lambda x: x.iloc[:-1])

    return filtered_data

In [ ]:
def correct_for_ohmic_resistance(
    df: pd.DataFrame,
    ohmic_resistance: float,
    ohmic_correction_factor: float = OHMIC_CORRECTION_FACTOR,
) -> pd.DataFrame:
    """Correct potential for ohmic resistance

    Args:
        df (pd.DataFrame): Dataframe containing the data to correct. Must contain a
        column named "Current [A]" and a column named "Potential (WE vs. RHE) [V]".
        ohmic_resistance (float): Ohmic resistance in ohm

    Returns:
        pd.DataFrame: Dataframe with corrected potential
    """
    print("Correcting platinum scan for ohmic resistance")
    df["Corrected potential (WE vs. RHE) [V]"] = (
        df["Potential (WE vs. RHE) [V]"]
        - ohmic_correction_factor * ohmic_resistance * df["Current [A]"]
    )
    return df

In [ ]:
def smooth_data_savitzky_golay(df: pd.DataFrame, window: float = 75):
    """Smooth data using Lowess smoothing

    Args:
        df (pd.DataFrame): Dataframe containing the data to smooth. Must contain a
        column named "Current [A]" and a column named "Corrected potential (WE vs. RHE) [V]".
        window (float, optional): Window size for smoothing. Defaults to 75.

    Returns:
        pd.DataFrame: Dataframe with smoothed data
        peak_value (float): Peak value of the smoothed data
    """
    print("Smoothing CV data")
    # Smooth data using Savitzky-Golay filter
    if len(df) < window:
        pass
    else:
        smoothed_current = savgol_filter(df["Current [A]"], window, 3)
        df["Current [A]"] = smoothed_current
    # Select part of the data
    df = filter_data(
        df,
        lower_potential=0.65,
        upper_potential=0.85,
        first_cycle=19,
        last_cycle=19,
        apply_row_removal_for_prettyness=False,
    )

    # Find the maximum value of the smoothed data (pandas data series)
    print("Finding platinum oxidation peak in selected potential window")
    peak_current = df["Current [A]"].max()

    # Find the potential at which the peak value occurs
    print("Finding corresponding potential at which the peak in current occurs")
    peak_value = df[df["Current [A]"] == peak_current][
        "Corrected potential (WE vs. RHE) [V]"
    ].values[0]

    df["Category"] = f"Fit: smoothed_data pt_peak = {round(peak_value, 3)} V"

    return df, peak_value

In [ ]:
def get_platinum_potential(df_pt: pd.DataFrame, ohmic_resistance: float):
    """Measure the potential of the platinum electrode

    Args:
        ohmic_resistance (float): Ohmic resistance of the platinum electrode

    Returns:
        df_pt_smoothed (pd.DataFrame): Smoothed data of the platinum electrode
        pt_peak_potential_ohmic_corrected (float): Peak potential of the platinum electrode
        df_pt (pd.DataFrame): Original data of the platinum electrode
    """

    # Set column headers
    df_pt = set_column_headers_cv(df_pt)
    print(f"df_pt: {df_pt.to_string}")

    # Correct for ohmic resistance
    df_pt = correct_for_ohmic_resistance(df_pt, ohmic_resistance, OHMIC_CORRECTION_FACTOR)

    # Select part of the data
    df_pt_filtered = filter_data(
        df_pt,
        lower_potential=0.65,
        upper_potential=1.2,  # It is cut further in the smoothing function
        first_cycle=19,
        last_cycle=19,
    )
    print(f"df_pt: {df_pt_filtered.to_string}")

    # Smooth data
    # df_pt_smoothed, pt_peak_potential_ohmic_corrected = smooth_data_lowess(df_pt_filtered)
    df_pt_smoothed, pt_peak_potential_ohmic_corrected = smooth_data_savitzky_golay(df_pt_filtered)
    print(f"pt_peak_potential_ohmic_corrected: {pt_peak_potential_ohmic_corrected}")
    print(f"df_pt_smoothed: {df_pt_smoothed.to_string}")

    # Give original a name in the column "Category"
    df_pt["Category"] = "Measurement"

    # Merge data
    # logging.debug("Merging smoothed CV data and original ")
    # df_merged = pd.concat([df_pt, df_pt_smoothed], ignore_index=True)

    return df_pt_smoothed, pt_peak_potential_ohmic_corrected, df_pt